# Reconstructing a midi from an instrument in transposed piano roll form

### Using the dataloader exclusively to generate a sample that looks like an output from the network for now

In [ ]:
# Add the src folder to the path
import numpy as np
import pandas as pd
import sys
import pretty_midi
sys.path.insert(0, '../src/')

import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display as display

from data.dataloader import MidiDataset

In [ ]:
midi_dataset = MidiDataset(csv_file='./piano_rolls.csv')

In [ ]:
sample = midi_dataset.__getitem__(0)['piano_rolls']
sample

The samples I have locally aren't normalized to 1 for the note velocities, so I'm just doing that first

In [ ]:
np.unique(sample)

In [ ]:
sample = sample.clip(0,1)

In [ ]:
np.unique(sample)

Good. Now let's pretend this is a network output and try to convert this back into a MIDI file

In [ ]:
sample

In [ ]:
# Figure out a way to get this so it's in line with the columns of the preprocessed data
column_names = [pretty_midi.note_number_to_name(n) for n in range(48,109)]
piano_roll = pd.DataFrame(sample, columns=column_names)

#piano_roll

In [ ]:
def midi_from_piano_roll(pr, tempo = 120):
    """
        We're taking some assumptions here to reconstruct the midi.
    """
    program = 0
    velocity = 42.
    bps = tempo / 60
    sps = bps * 4 # sixteenth notes per second
    
    # Create a PrettyMIDI object
    piano_midi = pretty_midi.PrettyMIDI()

    piano = pretty_midi.Instrument(program=program)
    # Iterate over note names, which will be converted to note number later
    for idx in pr.index:
        for note_name in pr.columns:
            #print(note_name)

            # Check if the note is activated at this timestep
            if pr.iloc[idx][note_name] == 1.:
                # Retrieve the MIDI note number for this note name
                note_number = pretty_midi.note_name_to_number(note_name)
                
                note_start = idx/sps # 0 if tempo = 60
                note_end = (idx+1)/sps # 0.25

                # Create a Note instance, starting according to the timestep * 16ths, ending one sixteenth later
                note = pretty_midi.Note(
                    velocity=velocity, pitch=note_number, start=note_start, end=note_end)
                # Add it to our cello instrument
                piano.notes.append(note)
    # Add the cello instrument to the PrettyMIDI object
    piano_midi.instruments.append(piano)
    return piano_midi
    # Write out the MIDI data
    #piano_midi.write('cello-C-chord.mid')

In [ ]:
midi = midi_from_piano_roll(piano_roll)

In [ ]:
midi.get_piano_roll()

In [ ]:
plt.figure(figsize=(10, 3))
display.specshow(midi.get_piano_roll(), y_axis='cqt_note', cmap=plt.cm.hot)